In [12]:
import pandas as pd
import numpy as np
import os 
import glob
from PIL import Image
import cv2
from pathlib import Path
import torch
import matplotlib.pyplot as plt
from functions import *
import mpimg
import matplotlib.image as mpimg  # Ensure correct import
from NN import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# path = '/Users/maelynenguyen/Desktop/79ca5c85-0d00-45fd-81e9-aa787898ebf5_epfml-segmentation/training/images/'
path = os.getcwd() + '/dataset/training/images/'

In [65]:
data = extract_data(path, 100, 16)

Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_001.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_002.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_003.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_004.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_005.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_006.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_007.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_008.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_009.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_010.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_011.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_012.png
Loading /Users/m

In [64]:
labels = extract_labels(path,100,16)

Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_001.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_002.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_003.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_004.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_005.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_006.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_007.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_008.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_009.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_010.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_011.png
Loading /Users/melina/Desktop/ML/MLproj2/dataset/training/images/satImage_012.png
Loading /Users/m

In [66]:
# split data into training and validation
train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))
test_size = int(0.1 * len(data))

train_data = data[:train_size]
train_labels = labels[:train_size]
val_data = data[train_size:train_size+val_size]
val_labels = labels[train_size:train_size+val_size]
test_data = data[train_size+val_size:]
test_labels = labels[train_size+val_size:]


In [67]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset


# Dataset and balancing
c0, c1 = 0, 0
for label in train_labels:
    if label[0] == 1:
        c0 += 1
    else:
        c1 += 1
print(f"Number of data points per class: c0 = {c0}, c1 = {c1}")

print("Balancing training data...")
min_c = min(c0, c1)
idx0 = [i for i, label in enumerate(train_labels) if label[0] == 1]
idx1 = [i for i, label in enumerate(train_labels) if label[1] == 1]
new_indices = idx0[:min_c] + idx1[:min_c]

train_data = train_data[new_indices]
train_labels = train_labels[new_indices]

c0, c1 = 0, 0
for label in train_labels:
    if label[0] == 1:
        c0 += 1
    else:
        c1 += 1
print(f"Number of data points per class after balancing: c0 = {c0}, c1 = {c1}")

Number of data points per class: c0 = 15831, c1 = 34169
Balancing training data...
Number of data points per class after balancing: c0 = 15831, c1 = 15831


In [106]:
# Model definition
class SimpleCNN(nn.Module):
    def __init__(self, num_channels, img_patch_size, num_labels):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, out_channels=16, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=2)
        self.fc1 = nn.Linear((img_patch_size // 4) * (img_patch_size // 4) * 32, 512)
        self.fc2 = nn.Linear(512, num_labels)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten for fully connected layers
        # x = x.reshape(x.size(0), -1)  # Use reshape instead of view

        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [69]:
# Hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 10
LEARNING_RATE = 0.01
SEED = 42
NUM_CHANNELS = 3  # Example, adjust as needed
IMG_PATCH_SIZE = 16
NUM_LABELS = 2

# DataLoader
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_data), torch.tensor(train_labels))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [70]:
train_data.shape, train_labels.shape

((31662, 16, 16, 3), (31662, 2))

In [71]:
# Model, loss, and optimizer
torch.manual_seed(SEED)
model = SimpleCNN(NUM_CHANNELS, IMG_PATCH_SIZE, NUM_LABELS)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels.argmax(dim=1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Loss: {running_loss / len(train_loader)}")

print("Training complete.")


Epoch 1/10, Loss: 0.14349827622086977
Epoch 2/10, Loss: 0.07340081940113151
Epoch 3/10, Loss: 0.05299746775335275
Epoch 4/10, Loss: 0.05045303524230169
Epoch 5/10, Loss: 0.04179121173817266
Epoch 6/10, Loss: 0.04371056631360059
Epoch 7/10, Loss: 0.036924062809267716
Epoch 8/10, Loss: 0.038400296942808096
Epoch 9/10, Loss: 0.0364379816507176
Epoch 10/10, Loss: 0.030370681660979146
Training complete.


In [77]:
test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_data), torch.tensor(test_labels))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [74]:
test_data.shape, test_labels.shape

((6250, 16, 16, 3), (6250, 2))

In [86]:
PIXEL_DEPTH = 255

# Write predictions from neural network to a file
def write_predictions_to_file(predictions, labels, filename):
    max_labels = np.argmax(labels, 1)
    max_predictions = np.argmax(predictions, 1)
    file = open(filename, "w")
    n = predictions.shape[0]
    for i in range(0, n):
        file.write(max_labels(i) + " " + max_predictions(i))
    file.close()


# Print predictions from neural network
def print_predictions(predictions, labels):
    max_labels = np.argmax(labels, 1)
    max_predictions = np.argmax(predictions, 1)
    print(str(max_labels) + " " + str(max_predictions))


# Convert array of labels to an image
def label_to_img(imgwidth, imgheight, w, h, labels):
    array_labels = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0, imgheight, h):
        for j in range(0, imgwidth, w):
            if labels[idx][0] > 0.5:  # bgrd
                l = 0
            else:
                l = 1
            array_labels[j : j + w, i : i + h] = l
            idx = idx + 1
    return array_labels


def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * PIXEL_DEPTH).round().astype(np.uint8)
    return rimg


def concatenate_images(img, gt_img):
    n_channels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if n_channels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)
        gt_img_3c[:, :, 0] = gt_img8
        gt_img_3c[:, :, 1] = gt_img8
        gt_img_3c[:, :, 2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg


def make_img_overlay(img, predicted_img):
    w = img.shape[0]
    h = img.shape[1]
    color_mask = np.zeros((w, h, 3), dtype=np.uint8)
    color_mask[:, :, 0] = predicted_img * PIXEL_DEPTH

    img8 = img_float_to_uint8(img)
    background = Image.fromarray(img8, "RGB").convert("RGBA")
    overlay = Image.fromarray(color_mask, "RGB").convert("RGBA")
    new_img = Image.blend(background, overlay, 0.2)
    return new_img

In [104]:


# Assuming `model` is your trained PyTorch model

def get_prediction(img):
    # Convert the input image to a NumPy array (if not already in this format)
    data = np.asarray(img_crop(img, IMG_PATCH_SIZE, IMG_PATCH_SIZE))
    
    # Convert NumPy array to PyTorch tensor, add batch dimension, and normalize if needed
    # data_tensor = torch.tensor(data, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    
    # Ensure the tensor is on the same device as the model (GPU/CPU)

    # data_loader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True)
    
    # data_tensor = data_loader.to(device)
    
    # Get the model's output (no need to apply softmax manually, PyTorch's CrossEntropyLoss handles this internally)
    output = model(data)
    
    # Apply softmax for prediction probabilities (optional, depending on your needs)
    softmax_output = torch.softmax(output, dim=1)
    
    # Get the prediction (index of the highest probability class)
    output_prediction = softmax_output.argmax(dim=1).cpu().numpy()  # Move to CPU and convert to numpy
    
    # Convert the prediction to an image (adjust according to your label_to_img function)
    img_prediction = label_to_img(img.shape[0], img.shape[1], IMG_PATCH_SIZE, IMG_PATCH_SIZE, output_prediction)
    
    return img_prediction


In [88]:
def get_prediction_with_groundtruth(filename, image_idx):
    imageid = "satImage_%.3d" % image_idx
    image_filename = filename + imageid + ".png"
    
    # Read the image
    img = plt.imread(image_filename)  # Use plt.imread for image loading
    
    # Get the prediction
    img_prediction = get_prediction(img)
    
    # Concatenate prediction with ground truth
    cimg = concatenate_images(img, img_prediction)  # Assuming concatenate_images works with the same input
    
    return cimg


In [89]:
def get_prediction_with_overlay(filename, image_idx):
    imageid = "satImage_%.3d" % image_idx
    image_filename = filename + imageid + ".png"
    
    # Read the image
    img = plt.imread(image_filename)
    
    # Get the prediction
    img_prediction = get_prediction(img)
    
    # Create the overlay (you may need to define make_img_overlay if it doesn't exist)
    oimg = make_img_overlay(img, img_prediction)
    
    return oimg


In [107]:
# Set model to evaluation mode
model.eval()

# Initialize variables for evaluation metrics
correct = 0
total = 0
running_loss = 0.0

# Disable gradient calculation during testing
with torch.no_grad():
    for batch_idx, (data, labels) in enumerate(test_loader):
        
        outputs = model(data)
        loss = criterion(outputs, labels.argmax(dim=1))
        running_loss += loss.item()
        
        
        prediction_training_dir = "predictions_training/"
        TRAINING_SIZE = len(test_loader)
        train_data_filename = "dataset/training/images/"
        
        if not os.path.isdir(prediction_training_dir):
            os.mkdir(prediction_training_dir)
        for i in range(1, TRAINING_SIZE + 1):
            pimg = get_prediction_with_groundtruth(train_data_filename, i)
            Image.fromarray(pimg).save(
                prediction_training_dir + "prediction_" + str(i) + ".png"
            )
            oimg = get_prediction_with_overlay(train_data_filename, i)
            oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")
        # Get predicted labels
        # _, predicted = torch.max(outputs, 1)  # Choose the class with the highest output value
        
        # # Update correct predictions count
        # total += labels.size(0)
        # correct += (predicted == labels).sum().item()

# Calculate accuracy and average loss
# accuracy = 100 * correct / total
average_loss = running_loss / len(test_loader)

# print(f"Test Accuracy: {accuracy:.2f}%")
print(f"Test Loss: {average_loss:.4f}")

AttributeError: 'numpy.ndarray' object has no attribute 'permute'